In [105]:
# import necessary objects and modules

from ROOT import TString, FairRunAna, FairFileSource, FairParRootFileIo, FairParAsciiFileIo
from ROOT import RhoMassParticleSelector, PndAnalysis, TFile,  RhoCandList, PndAnalysis
import numpy as np


prefix = "signal"
nevts = 0

OutFile = "Out_dummy.root"

inPidFile = prefix+"_pid.root"
inParFile = prefix+"_par.root"
pidParFile = TString('/home/lewis/fairsoft_may18/pandaroot/macro/params/all.par')

In [2]:
# set objects and input/output files

fRun = FairRunAna()
rtdb = fRun.GetRuntimeDb()
source = FairFileSource(inPidFile)
source.AddFriend("signal_full_nn.root")
fRun.SetSource(source)

parIO = FairParRootFileIo()
parIO.open(inParFile)
parIOPid = FairParAsciiFileIo()
parIOPid.open(pidParFile.Data(),"in")

rtdb.setFirstInput(parIO)
rtdb.setSecondInput(parIOPid)
rtdb.setOutput(parIO)

fRun.SetOutputFile(OutFile)
fRun.Init()
out = TFile.Open(prefix+"_conventional","RECREATE")

[WARN] FairRun::SetOutputFile() deprecated. Use FairRootFileSink.
[INFO] The input consists out of the following trees and files: 
[INFO]  - pndsim
[INFO]     - signal_pid.root
[INFO]  - FriendTree_1
[INFO]     - signal_full_nn.root
[INFO] FairRootFileSink initialized.
[INFO]  - cbmout
[INFO]     - Out_dummy.root
[INFO] Parameter and input file are available, Assure that basic info is there for the run!

************************************************************* 
     initialisation for run id 1564673049
************************************************************* 
-I- FairRunTimeDB::InitContainer() FairGeoParSet
Container FairGeoParSet initialized from ROOT file.
-I- FairRunTimeDB::InitContainer() FairBaseParSet
Info in (PndGeoHandling::Instance): Making a new instance using the framework.
Container FairBaseParSet initialized from ROOT file.

************************************************************* 
     initialisation for run id 1564673049
***********************************

Info in <TGeoManager::CloseGeometry>: Geometry loaded from file...
Info in <TGeoManager::SetTopVolume>: Top volume is cave. Master volume is cave
Info in <TGeoNavigator::BuildCache>: --- Maximum geometry depth set to 100
Info in <TGeoManager::Voxelize>: Voxelizing...
Info in <TGeoManager::CountLevels>: max level = 12, max placements = 4550
Info in <TGeoManager::CloseGeometry>: 2028308 nodes/ 5654 volume UID's in FAIR geometry
Info in <TGeoManager::CloseGeometry>: ----------------modeler ready----------------
Warning in <TTree::Bronch>: FairEventHeader cannot be split, resetting splitlevel to 0


In [3]:
# this analysis is being done for the signal_pid.root file generated from the rho tutorial
# there are only muons and pions in these events

theAnalysis = PndAnalysis()
if (nevts==0):
    nevts= theAnalysis.GetEntries()

muon = RhoCandList()
pion = RhoCandList()

[INFO] Object PidNeutralCand describing the branch in the folder structure was found
[INFO]  Branch: PidChargedCandElectron not found in Tree.
  ####################  No PidChargedCand with Electron tracking hypothesis
-W- PndAnalysis::Init(): No barrel track inpt array.
-W- PndAnalysis::Init(): No forward track inpt array.
[INFO]  Branch: PidChargedCandMuon not found in Tree.
  ####################  No PidChargedCand with Muon tracking hypothesis
-W- PndAnalysis::Init(): No barrel track inpt array.
-W- PndAnalysis::Init(): No forward track inpt array.
[INFO]  Branch: PidChargedCandPion not found in Tree.
  ####################  No PidChargedCand with Pion tracking hypothesis
-W- PndAnalysis::Init(): No barrel track inpt array.
-W- PndAnalysis::Init(): No forward track inpt array.
[INFO]  Branch: PidChargedCandKaon not found in Tree.
  ####################  No PidChargedCand with Kaon tracking hypothesis
-W- PndAnalysis::Init(): No barrel track inpt array.
-W- PndAnalysis::Init(): No f

In [5]:
i = 0
num_classes = 5
y_test = []
y_score = []
y_score_ml = []

# iterate through each event for analysis
while theAnalysis.GetEvent():
    i += 1
    if ((i%10000)==0):
        print("evt ",i)
    #choose which particle, cut, and method to perform the analysis
    theAnalysis.FillList(muon,  "MuonAll",  "PidAlgoMl")
    for ii in range(muon.GetLength()):
        if not (theAnalysis.McTruthMatch(muon[ii])):
            prob = np.empty(num_classes)
            for jj in range(num_classes):
                prob[jj] = (muon[ii].GetPidInfo(jj))
            y_score_ml.append(prob) # append the probability to the y_score array
    
    #choose which particle, cut, and method to perform the analysis
    theAnalysis.FillList(pion,  "PionAll",  "PidAlgoMl")
    for ii in range(pion.GetLength()):
        if not (theAnalysis.McTruthMatch(pion[ii])): 
            prob = np.empty(num_classes)
            for jj in range(num_classes):
                prob[jj] = (pion[ii].GetPidInfo(jj))
            y_score_ml.append(prob) # append the probability to the y_score array
            
            
    #choose which particle, cut, and method to perform the analysis        
    theAnalysis.FillList(muon,  "MuonAll",  "PidAlgoMvd;PidAlgoStt;PidAlgoDrc;PidAlgoMdtHardCuts;PidAlgoDisc;PidAlgoEmcBayes;PidAlgoSciT;PidAlgoRich")
    for ii in range(muon.GetLength()):
        if not (theAnalysis.McTruthMatch(muon[ii])):
            prob = np.empty(num_classes)
            for jj in range(num_classes):
                prob[jj] = (muon[ii].GetPidInfo(jj))
            y_score.append(prob) # append the probability to the y_score array
            y_test.append(2) # append the true value of the pion to the array
    
    #choose which particle, cut, and method to perform the analysis
    theAnalysis.FillList(pion,  "PionAll",  "PidAlgoMvd;PidAlgoStt;PidAlgoDrc;PidAlgoMdtHardCuts;PidAlgoDisc;PidAlgoEmcBayes;PidAlgoSciT;PidAlgoRich")
    for ii in range(pion.GetLength()):
        if not (theAnalysis.McTruthMatch(pion[ii])): 
            prob = np.empty(num_classes)
            for jj in range(num_classes):
                prob[jj] = (pion[ii].GetPidInfo(jj))
            y_score.append(prob) # append the probability to the y_score array
            y_test.append(1) # append the true value of the pion to the array

            
# turn the values into a numpy array and remove any NaN values
y_test = np.asarray(y_test)
y_score = np.asarray(y_score)
y_score = np.nan_to_num(y_score,0.0)


y_score_ml = np.asarray(y_score_ml)
y_score_ml = np.nan_to_num(y_score_ml,0.0)

evt  10000
evt  20000
evt  30000
evt  40000
evt  50000
evt  60000
evt  70000
evt  80000
evt  90000
[INFO] Object PidAlgoMvd describing the branch in the folder structure was found
[INFO] Object PidAlgoStt describing the branch in the folder structure was found
[INFO] Object PidAlgoDrc describing the branch in the folder structure was found
[INFO] Object PidAlgoMdtHardCuts describing the branch in the folder structure was found
[INFO] Object PidAlgoDisc describing the branch in the folder structure was found
[INFO] Object PidAlgoEmcBayes describing the branch in the folder structure was found
[INFO] Object PidAlgoSciT describing the branch in the folder structure was found
[INFO] Object PidAlgoRich describing the branch in the folder structure was found


Info in <PndAnalysis::GetEvent()>: Maximum number of entries in the file chain reached: 100000.


In [6]:
print(y_test)
print(y_score)

[2 1 1 ... 2 2 1]
[[1.17900933e-048 9.51476573e-001 4.85234266e-002 0.00000000e+000
  0.00000000e+000]
 [0.00000000e+000 1.00000000e+000 0.00000000e+000 0.00000000e+000
  0.00000000e+000]
 [0.00000000e+000 1.00000000e+000 0.00000000e+000 0.00000000e+000
  0.00000000e+000]
 ...
 [3.26552695e-012 9.44624419e-007 9.61582547e-007 1.45120317e-001
  8.54877777e-001]
 [2.60028096e-029 9.99815666e-001 1.84333516e-004 0.00000000e+000
  0.00000000e+000]
 [5.51615297e-008 8.37690058e-001 1.62309887e-001 1.75533479e-101
  0.00000000e+000]]


In [7]:
# create the mask for the monte carlo truth matches for the particles
muon_mask = y_test == 1
pion_mask = y_test == 2

In [96]:
# Plot probability distributions for the muons

import matplotlib.pyplot as plt
%matplotlib tk

#plt.hist(y_score[muon_mask][:,1],50,(0,1),color='cornflowerblue')
plt.hist((y_score[muon_mask][:,1],y_score_ml[muon_mask][:,1]),50,(0,1),\
         color=['red','blue'],label=['Conventional Methods','Neural Network'])
plt.title('Probability Distribution for FTM Muons', fontsize=18)
plt.xlabel('Pid Probability', fontsize=16)
plt.ylabel('Counts', fontsize=16)
plt.yscale('log')
plt.xlim(0,1)
#plt.ylim(0,14000)
plt.tight_layout()
plt.legend(loc='best', fontsize=14)
#plt.xticks(np.arange(0, 1.1, step=0.1))
#plt.yticks(np.arange(0, 150000, step=20000))
plt.grid()
plt.savefig('probability_ftm_muons_both_log.png')
plt.show()

In [103]:
# Plot probability distributions for the other particles with the conventional method

colors = ['aqua','darkorange','black','green']
labels = ['$e^{-}$','$\pi^{-}$','$k^{-}$','$\overline{p}$']

plt.hist(np.delete(y_score, 1, axis=1)[muon_mask],10,(0.25,1),label=labels,color=colors)
plt.title('Probability Distribution for FTM Muons', fontsize=18)
plt.xlabel('Conventional Pid Probability', fontsize=16)
plt.ylabel('Counts', fontsize=16)
plt.tight_layout()
plt.xlim(0.25,1)
plt.grid()
plt.legend()
plt.savefig('probability_other_ftm_muons.png')
plt.show()

In [104]:
# Plot probability distributions for the other particles with the neural network

colors = ['aqua','darkorange','black','green']
labels = ['$e^{-}$','$\pi^{-}$','$k^{-}$','$\overline{p}$']

plt.hist(np.delete(y_score_ml, 1, axis=1)[muon_mask],10,(0.25,1),label=labels,color=colors)
plt.title('Probability Distribution for FTM Muons', fontsize=18)
plt.xlabel('Neural Network Pid Probability', fontsize=16)
plt.ylabel('Counts', fontsize=16)
plt.tight_layout()
plt.xlim(0.25,1)
plt.grid()
plt.legend()
plt.savefig('probability_other_ftm_muons_nn.png')
plt.show()

In [98]:
# Plot probability distributions for the pions

#plt.hist(y_score[pion_mask][:,2],50,(0,1),color='darkorange')
plt.hist((y_score[pion_mask][:,2],y_score_ml[pion_mask][:,2]),50,(0,1),\
         color=['red','blue'],label=['Conventional Methods','Neural Network'])
plt.title('Probability Distribution for FTM Pions', fontsize=18)
plt.xlabel('Pid Probability', fontsize=16)
plt.ylabel('Counts', fontsize=16)
plt.tight_layout()
plt.xlim(0,1)
plt.legend(loc='best', fontsize=14)
plt.grid()
#plt.yscale('log')
plt.savefig('probability_ftm_pions_both.png')
plt.show()

In [99]:
# Plot probability distributions for the other particles with the conventional method

colors = ['aqua','cornflowerblue','black','green']
labels = ['$e^{-}$','$\mu^{-}$','$k^{-}$','$\overline{p}$']

plt.hist(np.delete(y_score, 2, axis=1)[pion_mask],10,(0.25,1),label=labels,color=colors)
plt.title('Probability Distribution for FTM Pions', fontsize=18)
plt.xlabel('Conventional Pid Probability', fontsize=16)
plt.ylabel('Counts', fontsize=16)
plt.tight_layout()
plt.xlim(0.25,1)
plt.grid()
plt.legend()
plt.savefig('probability_other_ftm_pions.png')
plt.show()

In [100]:
# Plot probability distributions for the other particles

colors = ['aqua','cornflowerblue','black','green']
labels = ['$e^{-}$','$\mu^{-}$','$k^{-}$','$\overline{p}$']

plt.hist(np.delete(y_score_ml, 2, axis=1)[pion_mask],10,(0.25,1),label=labels,color=colors)
plt.title('Probability Distribution for FTM Pions', fontsize=18)
plt.xlabel('Neural Network Pid Probability', fontsize=16)
plt.ylabel('Counts', fontsize=16)
plt.tight_layout()
plt.xlim(0.25,1)
plt.grid()
plt.legend()
plt.savefig('probability_other_ftm_pions_nn.png')
plt.show()